In [1]:
from gazpacho import Soup
import pandas as pd
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import Select

In [2]:
options = Options()
options.headless = True

Binder/Laptop Configuration for Selenium:

In [3]:
binder = True

In [4]:
if binder:
    from webdriverdownloader import GeckoDriverDownloader
    gdd = GeckoDriverDownloader()
    gdd.download_and_install("v0.26.0")
    browser = Firefox(executable_path='/home/jovyan/bin/geckodriver', options=options)
else:
    browser = Firefox(executable_path='/usr/local/bin/geckodriver', options=options)

Okay!

In [5]:
url = "https://scrape.world/season"
browser.get(url)

#### Login

In [6]:
username = browser.find_element_by_id("username")
username.clear()
username.send_keys("admin")

password = browser.find_element_by_name("password")
password.clear()
password.send_keys("admin")

browser.find_element_by_xpath("/html/body/div/div/form/div/input[3]").click()

In [7]:
browser.get(url)
html = browser.page_source
soup = Soup(html)

In [8]:
str(soup)[4000:5000]

'ata-tip="Average points in simulated seasons">PTS</th>\n        </tr>\n    </thead>\n    <tbody>\n        <tr class="no_ranker partial_table">\n            <td colspan="18"><strong>Atlantic Division</strong></td>\n        </tr>\n        <tr>\n            <th scope="row" class="right " data-stat="ranker" csk="1">1</th>\n            <td class="left " data-stat="team_name" csk="111.6">Boston Kodiaks</td>\n            <td class="right " data-stat="wins_avg">48.5</td>\n            <td class="right " data-stat="losses_avg">18.8</td>\n            <td class="right " data-stat="losses_ot_avg">14.7</td>\n            <td class="right " data-stat="points_avg">111.6</td>\n        </tr>\n        <tr>\n            <th scope="row" class="right " data-stat="ranker" csk="2">2</th>\n            <td class="left " data-stat="team_name" csk="108.1">Tampa Bay Thunder</td>\n            <td class="right " data-stat="wins_avg">50.2</td>\n            <td class="right " data-stat="losses_avg">24.1</td>\n        

In [9]:
tables = pd.read_html(browser.page_source)

In [10]:
east = tables[0]
west = tables[1]

In [11]:
east.head()

,Rk,Team,W,L,OL,PTS,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division
1,1,Boston Kodiaks,48.5,18.8,14.7,111.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Tampa Bay Thunder,50.2,24.1,7.7,108.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Florida Jaguars,44.5,28.3,9.2,98.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df = pd.concat([east, west], axis=0)
df["W"] = df["W"].apply(pd.to_numeric, errors="coerce")
df = df.dropna(subset=["W"])
df = df[["Team", "W"]]
df = df.rename(columns={"Team": "team", "W": "wins"})
df = df.sort_values("wins", ascending=False)

In [13]:
df.tail()

,team,wins
17,Anaheim Mallards,35.1
19,New Jersey Demons,31.9
18,Los Angeles Monarchs,31.0
8,Ottawa Legislators,29.9
9,Detroit Carmine Feathers,21.7


#### JavaScript

In [14]:
url = "https://scrape.world/results"
browser.get(url)

In [15]:
search = browser.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/label/input")
search.clear()
search.send_keys("toronto")

In [16]:
drop_down = Select(
    browser.find_element_by_xpath("/html/body/div/div/div[2]/div[1]/label/select")
)
drop_down.select_by_visible_text("100")

In [17]:
html = browser.page_source
soup = Soup(html)
df = pd.read_html(str(soup.find("table")))[0]

In [18]:
df.tail(10)

,day,away,goals_away,home,goals_home,extra_time_loss
48,109,Chicago Greyfalcons,6,Toronto Pine Needles,2,0
49,118,Toronto Pine Needles,5,Nashville Carnivores,2,0
50,120,Toronto Pine Needles,5,Dallas Celebrities,3,0
51,123,Ottawa Legislators,1,Toronto Pine Needles,2,1
52,125,Florida Jaguars,5,Toronto Pine Needles,3,0
53,127,Toronto Pine Needles,3,New York Officials,5,0
54,129,Anaheim Mallards,4,Toronto Pine Needles,5,1
55,130,Toronto Pine Needles,1,Montreal Quebecers,2,1
56,133,Arizona Dingos,2,Toronto Pine Needles,3,1
57,135,Dallas Celebrities,3,Toronto Pine Needles,2,0
